#   _Key Derivation Functions_ - Funkcje wyprowadzania klucza
## LBK, Kollbi (Insert names later)
###   Podstawowe informacje
1.  Funkcje KDF to algorytmy kryptograficzne zwracające sekret lub wiele sekretów z tajnej wartości.    
        Np. Stworzenie klucza prywatnego (jeden sekret) z hasła użytkownika (tajna wartość).
1.  Metody te powstały ponieważ hasła użytkowników są przewidywalne i krótkie (niska entropia). Pojawia się potrzeba stworzenia mocnego klucza.  
Jednak nikt nie będzie pamiętał 64 znakowego hasła. Dlatego z pomocą przychodzi `key-stretching`.  
Proces w którym tworzony jest bezpieczniejszy i (teoretycznie) odporniejszy na ataki siłowe nowy klucz.  
`Key-streching` jest jedną z kluczowych technik zwiększania bezpieczeństwa klucza. Do key-streching'u można zaliczyć proste funkcje hashujące jak SHA-*, MD5 (nie używać!) lub szyfry blokowe.
1. Funkcje wyprowadzania klucza powinny być `deterministyczne`. Oznacza to, że dla tych samych danych wejściowych otrzymamy zawsze ten sam skrót niezależnie ile razy funkcja zostanie wywołana.
1. Key Derivation Functions mogą być używane nie tylko do zapisywania haseł w postaci skrótów w bazie danych. Można je wykorzystywać między innymi do symetrycznego szyfrowania dysków, kryptografi asymetrycznej oraz niektóre odnogi algorytmów znajdują swoje zastosowanie w technologiach `blockchain`.
```
                                           _          _  ______  _____                               _       
 _ __   __ _ ___ _____      _____  _ __ __| |        | |/ /  _ \|  ___|       ___  ___  ___ _ __ ___| |_ ___  
| '_ \ / _` / __/ __\ \ /\ / / _ \| '__/ _` | =====\ | ' /| | | | |_   ====\ / __|/ _ \/ __| '__/ _ \ __/ __|
| |_) | (_| \__ \__ \\ V  V / (_) | | | (_| | =====/ | . \| |_| |  _|  ====/ \__ \  __/ (__| | |  __/ |_\__ \
| .__/ \__,_|___/___/ \_/\_/ \___/|_|  \__,_|        |_|\_\____/|_|          |___/\___|\___|_|  \___|\__|___/
|_|                                                     
```

---

### Funkcje KDF to nie tylko funkcje haszujące! Jednak mają wspólną część
1. Tak samo jak funkcje haszujące szyfrują dane deterministycznie. (To samo wejście - to samo wyjście)
1. Na wyjściu jest zawsze ciąg o takiej samej długości. Bez znaczenia jak duże lub krótkie były dane wejściowe.
1. Działa tylko w jedną stronę. Nie odzyska wejścia danych z wyjścia.


### Zrozumienie od podstaw

Najprostszą funkcję wyprowadzenia klucza można stworzyć wykorzystując dowolny algorytm haszujący. Dla przykładu użyty zostanie algorytm SHA256.  

function(`password`, `sha256`) --> `key`

In [1]:
from hashlib import sha256

hash = sha256(b"password")

print(hash.digest().hex())

5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8


Należy pamiętać, że powyższe rozwiązanie nie jest bezpieczne. Proste hasze są podatne na ataki słownikowe (np. tęczowe tablice (Rainbow table)). Jak zatem zwiększyć bezpieczeństwo?

Pierwszym krokiem jest dodanie `soli` (ang. `salt`).  
`Salt` jest wartością przechowywaną razem z uzyskanym kluczem. Używana, żeby utrudnić atak siłowy (np. Rainbow Table). Sprawia, że nowy hasz jest bardziej skomplikowany i zmniejsza szansę pojawienia się go we wcześniej obliczonych tęczowych tablicach.

function_with_salt(`salt`, `password`, `sha256`) --> `key`  

In [2]:
password = b"password"
salt = b"salt"
hash = sha256(salt+password)

print(hash.digest().hex())

13601bda4ea78e55a07b98866d2be6be0744e3866f13c00c811cab608a28f322


Funkcję wyżej można opisać jako `HKDF` (`HMAC-based key derivation function`) - najprostsza funkcja wyprowadzania klucza.  
Ciągle jest mniej bezpieczna niż aktualne KDF. Zaleca się korzystanie z PBKDF2, Bcrypt, Scrypt (ten projekt) oraz Argon2.



### Przykład
Wykorzystanie `HMAC` (Hash-based message authentication code) z solą, hasłem użytkownika oraz funkcją SHA256 do pozyskania klucza.
1. Sprawdź co się stanie jak zmienisz sól. Może to być bardzo mała zmiana. Porównaj parę wyników dla tego samego hasła. 
1. Czy klucze z nową solą znacznie różnią się od głównego? Jeśli tak to dlaczego. ([Lawinowość](https://en.wikipedia.org/wiki/Avalanche_effect))
1. Porównaj dwa klucze z tym samym hasłem w następujący sposób:  
    1. Dla samej funkcji haszującej `sha256(b"password")`
    1. Następnie stwórz dwie różne sole.
    1. Porównaj klucze dołączając sól po haśle `sha256(b"password+salt")`

In [3]:
from hmac import HMAC
from hashlib import sha256

salt = b'NaCl'
password = b'CryptoIsFun'

key = HMAC(salt, password, digestmod=sha256)

print(key.digest().hex())  # Desired output: `bff0e9482b9412e77f6b401c421957921782ff2f385521498ce59357fdc6da72`

bff0e9482b9412e77f6b401c421957921782ff2f385521498ce59357fdc6da72


### Zasosowanie `soli`
Wielokrotne użycie soli dla tego samego hasła umożliwia pozyskanie wielu innych kluczy. Jest to często stosowane rozwiązanie.  
Dla przykładu: można posiadać w bazie dwóch użytkowników z takim samym hasłem. Zwykłe zahaszowanie hasła w obu przypadkach zwróci to samo.  
Jednak wygenerowanie dla każdego z nich unikalnej wartości (`salt`) i dołączenie do hasła, sprawia że otrzymane hasze są diametralnie różne. ([Lawinowość](https://en.wikipedia.org/wiki/Avalanche_effect))  
KDF pomimo znajomości klucza z `salt1` nie jest wstanie znaleźć klucza z `salt2`.

### Po podstawach

Wiedząc co rozróżnia funkcje haszujące od KDF oraz czym jest sól, zwrócona zostanie uwaga na to, co powinna zapewniać dobra funkcja wyprowadzania klucza.  

1. Memory Hardness - cecha funkcji zapewniająca, że wymaga dużej ilości pamięci do obliczeń lub wymaga znacznie większego nakładu czasowego, tak by maszyny wykonujące obliczenia równoległe nie były znacząco bardziej efektywne w obliczeniach od standardowych procesorów CPU.  
Algorytm określa się "memory-hard" gdy potrzeba wysokiej ilości pamięci do pojedynczego wykonania. 

1. Funkcja KDF, która ma być wysoce odporna na ataki siłowe lub słownikowe, powinna maksymalnie spowolnić możliwość obliczeń równoległych. Jednocześnie wykonując relatywnie szybkie obliczenie klucza bądź skrótu.

1. Żeby "opóźnić" generację kluczy można zwiększyć złożoność obliczeniową, złożoność pamięciową lub oba jednocześnie.

Więcej informacji: `Funkcje wyprowadzania klucza (KDF) wykorzystujące odwzorowanie logistyczne - Grzegorz Frejek`

---

# Wprowadzenie `scrypt`

`Scrypt` jest funkcją wyprowadzania klucza opartą na haśle. Implementacja opiera się na funkcjach, które zapewniają dodatkową ochronę (memory hardness).

Scrypt posiada więcej parametrów niż stworzony przez nas wcześniej przykład. 
Parametry scrypt są następujące:
1. `Passphrase`
1. `Salt`
1. `Block size`
1. `CPU/Memory cost` (>1; potęga 2; < 2^(128 * r / 8))
1. `Parallelization parametr` (>0; <= ((2^32-1) * 32) / (128 * r))
1. `Key length` (długość w oktetach; >0; <= (2^32 - 1) * 32)

Najpierw należy zaznajomić się z poszczególnymi funkcjami algorytmu. A potem działanie algorytmu można przedstawić w 3 krokach.

### Salsa20/8 - wariant z [Salsa20](https://en.wikipedia.org/wiki/Salsa20)
Autor: Daniel J. Bernstein

`Salsa20/8` oraz `Salsa20/12` są wariantami z `Salsa20`. Nie zostały stworzone, żeby wyprzec oryginalną implementację. Wręcz przeciwnie. Mają za zadanie dopełniać i działać lepiej.  
Salsa20/8 nie może być uznana za funkcję haszującą ponieważ nie jest odporna na kolizje.

Aktualnie nie jest znana skuteczna metoda kryptoanalizy pełnej 20-rundowej Salsy (na 2015 rok). Wykryto wersje ataku na wersje z mniejszą liczbą rund. Szybszą kuzynką Salsy20 jest `ChaCha20` - miesza lepiej bity ale nie została przebadana jeszcze tak dobrze jak Salsa20.

Algorytm wykonuje operacje `XOR` na ciągach znaków w określonej liczbie rund. Salsa20 wykonuje 20 rund. Salsa20/8 i Salsa20/12 odpowiednio po 8 i 12 rund.

<!-- Można wstawić jakiś ładny opis. Troszkę bardziej rozbudowany. Co to robi czy coś. Chociaż kod mówi sam za siebie. -->
Implementacja Salsy20 w pythonie znajduje się poniżej.
<!-- Ja to bym wywalił wszystko do pythona. Potencjalne funkcje są w pliku salsa20.py albo salsa20_raw.py -->

Implementacja jest wzorowana na [link](https://github.com/Daeinar/salsa20).


Najistotniejszym elementem jest funkcja Quarter-Round. Następuje w niej xorowanie odpowiednio najpierw czterech kolumn. Potem transponujemy całą tablicę i przystępujemy do xorowania czterech wierszy. 

UWAGA: Kolejność operacji w funkcji quarter_round() ma znaczenie. Muszą one odbywać się sekwencyjnie po sobie.

In [4]:
# Pomocnicza funkcja - lewe przesunięcie cykliczne
def circular_left_shift(var, shift):
    return var[shift:] + var[:shift]

# Pomocnicza funkcja - alternatywa rozłączna
def XOR(bits1, bits2):
    xor_res = ""
    for bit1, bit2 in zip(bits1, bits2):
        xor_res += '0' if bit1 == bit2 else '1'
    return xor_res

# Pomocnicza funkcja - zrównanie liczb przez wydłużenie krótrszej dokładając zera z przodu
def make_equal_length(str1, str2):
    len1 = len(str1) 
    len2 = len(str2)
    if len1 < len2: 
        str1 = (len2 - len1) * '0' + str1 
        len1 = len2
    elif len2 < len1: 
        str2 = (len1 - len2) * '0' + str2 
    len2 = len1
    return len1, str1, str2

# Pomocnicza funkcja - dodawanie bitowe zachowujące długość liczby
def bin_add(bits1, bits2):
    result = ''
    length, first, second = make_equal_length(bits1, bits2) 
    carry = 0
    for i in range(length - 1, -1, -1): 
        firstBit = int(first[i]) 
        secondBit = int(second[i])
        sum = (firstBit ^ secondBit ^ carry) + 48
        result = chr(sum) + result
        carry = (firstBit & secondBit) | (secondBit & carry) | (firstBit & carry)
    return result

# Funkcja "Ćwierć-Rundy"
# Ciąg zawartych w niej operacji jest celowo nieodwracalny
def quarter_round(y0, y1, y2, y3):
    z1 = XOR(y1, circular_left_shift(bin_add(y0, y3), 7))
    z2 = XOR(y2, circular_left_shift(bin_add(z1, y0), 9))
    z3 = XOR(y3, circular_left_shift(bin_add(z2, z1), 13))
    z0 = XOR(y0, circular_left_shift(bin_add(z3, z2), 18))
    return z0, z1, z2, z3

# test 0
ty0 = "00000000000000000000000000000000"
ty1 = "00000000000000000000000000000000"
ty2 = "00000000000000000000000000000000"
ty3 = "00000000000000000000000000000000"
print(quarter_round(ty0, ty1, ty2, ty3))
# oczekiwany wynik:
# ( '00000000000000000000000000000000', '00000000000000000000000000000000',
#   '00000000000000000000000000000000', '00000000000000000000000000000000')

# test 1
ty0 = "11100111111010001100000000000110"
ty1 = "11000100111110010100000101111101"
ty2 = "01100100011110011011010010110010"
ty3 = "01101000110001100111000100110111"
print(quarter_round(ty0, ty1, ty2, ty3))
# oczekiwany wynik:
# ( '11101000011101101101011100101011', '10010011011000011101111111010101',
#   '11110001010001100000001001000100', '10010100100001010100000110100011')

('00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000')
('11101000011101101101011100101011', '10010011011000011101111111010101', '11110001010001100000001001000100', '10010100100001010100000110100011')


Kilka funkcji pomocniczych

In [5]:
def xor_block(b1, b2):
    return [b1[i]^b2[i] for i in range(16)]

def message_to_blocks(message):
    message_blocks = []
    for b in range(int(len(message)/64)):
        message_blocks.append([littleendian(message[64*b + 4*i : 4*i + 4 + 64*b]) for i in range(16)])
    return message_blocks


Każda runda to wykonanie operacji quarter_round() na kolumnach i wierszach.

Warto zwrócić uwagę, że Runda Kolumn jest ekwiwalentem ponownego wykonania Rundy Rzędów na macierzy transponowanej.

In [6]:
from copy import deepcopy

# Funkcja "Rundy Rzędów"
# jako wejście przyjmuje 16-blokową sekwencję
def row_round(y):
    y = list(y)
    # odpowiedź również będzie 16-blokową sekwencją
    z = deepcopy(y)
    z[0],   z[1],   z[2],   z[3]    =   quarter_round(y[0],  y[1],  y[2],  y[3])
    z[5],   z[6],   z[7],   z[4]    =   quarter_round(y[5],  y[6],  y[7],  y[4])
    z[10],  z[11],  z[8],   z[9]    =   quarter_round(y[10], y[11], y[8],  y[9])
    z[15],  z[12],  z[13],  z[14]   =   quarter_round(y[15], y[12], y[13], y[14])
    return z

ty = [
    "00000000000000000000000000000001", "00000000000000000000000000000000",
    "00000000000000000000000000000000", "00000000000000000000000000000000",
    "00000000000000000000000000000001", "00000000000000000000000000000000",
    "00000000000000000000000000000000", "00000000000000000000000000000000",
    "00000000000000000000000000000001", "00000000000000000000000000000000",
    "00000000000000000000000000000000", "00000000000000000000000000000000",
    "00000000000000000000000000000001", "00000000000000000000000000000000",
    "00000000000000000000000000000000", "00000000000000000000000000000000",
    ]

print(row_round(ty))
# oczekiwany wynik:
# [ '00001000000000001000000101000101', '00000000000000000000000010000000',
#   '00000000000000010000001000000000', '00100000010100000000000000000000',
#   '00100000000100000000000000000001', '00000000000001001000000001000100',
#   '00000000000000000000000010000000', '00000000000000010000000000000000',
#   '00000000000000000000000000000001', '00000000000000000010000000000000',
#   '10000000000001000000000000000000', '00000000000000000000000000000000',
#   '00000000000000000000000000000001', '00000000000000000000001000000000',
#   '00000000010000000010000000000000', '10001000000000000000000100000000']

# Funkcja "Rundy Kolumn"
# jako wejście przyjmuje 16-blokową sekwencję
def col_round(x):
    x = list(x)
    # odpowiedź również będzie 16-blokową sekwencją
    y = deepcopy(x)
    y[0],   y[4],   y[8],   y[12]   =   quarter_round(x[0],  x[4],  x[8],  x[12])
    y[5],   y[9],   y[13],  y[1]    =   quarter_round(x[5],  x[9],  x[13], x[1])
    y[10],  y[14],  y[2],   y[6]    =   quarter_round(x[10], x[14], x[2],  x[6])
    y[15],  y[3],   y[7],   y[11]   =   quarter_round(x[15], x[3],  x[7],  x[11])
    return y

tx = [
    "00000000000000000000000000000001", "00000000000000000000000000000000",
    "00000000000000000000000000000000", "00000000000000000000000000000000",
    "00000000000000000000000000000001", "00000000000000000000000000000000",
    "00000000000000000000000000000000", "00000000000000000000000000000000",
    "00000000000000000000000000000001", "00000000000000000000000000000000",
    "00000000000000000000000000000000", "00000000000000000000000000000000",
    "00000000000000000000000000000001", "00000000000000000000000000000000",
    "00000000000000000000000000000000", "00000000000000000000000000000000",
    ]

print(col_round(tx))
# oczekiwany wynik:
# [ '00010000000010010000001010001000', '00000000000000000000000000000000',
#   '00000000000000000000000000000000', '00000000000000000000000000000000',
#   '00000000000000000000000100000001', '00000000000000000000000000000000',
#   '00000000000000000000000000000000', '00000000000000000000000000000000',
#   '00000000000000100000010000000001', '00000000000000000000000000000000',
#   '00000000000000000000000000000000', '00000000000000000000000000000000',
#   '01000000101000000100000000000001', '00000000000000000000000000000000',
#   '00000000000000000000000000000000', '00000000000000000000000000000000']

['00001000000000001000000101000101', '00000000000000000000000010000000', '00000000000000010000001000000000', '00100000010100000000000000000000', '00100000000100000000000000000001', '00000000000001001000000001000100', '00000000000000000000000010000000', '00000000000000010000000000000000', '00000000000000000000000000000001', '00000000000000000010000000000000', '10000000000001000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000001', '00000000000000000000001000000000', '00000000010000000010000000000000', '10001000000000000000000100000000']
['00010000000010010000001010001000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000100000001', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000100000010000000001', '00000000000000000000000000000000', '00000000000000000000000000000000', '0000000000000000000000000

Teraz możemy powyższe funkcje złożyć w pełną rundę

In [7]:
# Funkcja Rundy
def double_round(x):
    # przygotowanie naszych zmiennych pomocniczych
    y = deepcopy(x)
    z = deepcopy(x)

    # pełna funkcja rundy to nic innego jak wykonanie funkcji kolumnowej,
    # a następnie rzędowej na tej samej tablicy słów
    y = col_round(x)
    z = row_round(y)

    return z

tx = [
    "00000000000000000000000000000001", "00000000000000000000000000000000",
    "00000000000000000000000000000000", "00000000000000000000000000000000",
    "00000000000000000000000000000000", "00000000000000000000000000000000",
    "00000000000000000000000000000000", "00000000000000000000000000000000",
    "00000000000000000000000000000000", "00000000000000000000000000000000",
    "00000000000000000000000000000000", "00000000000000000000000000000000",
    "00000000000000000000000000000000", "00000000000000000000000000000000",
    "00000000000000000000000000000000", "00000000000000000000000000000000",
    ]

print(double_round(tx))
# oczekiwany wynik:
# [ '10000001100001101010001000101101', '00000000010000001010001010000100',
#   '10000010010001111001001000010000', '00000110100100101001000001010001',
#   '00001000000000000000000010010000', '00000010010000000010001000000000',
#   '00000000000000000100000000000000', '00000000100000000000000000000000',
#   '00000000000000010000001000000000', '00100000010000000000000000000000',
#   '00001000000000001000000100000100', '00000000000000000000000000000000',
#   '00100000010100000000000000000000', '10100000000000000000000001000000',
#   '00000000000010000001100000001010', '01100001001010101000000000100000']

['10000001100001101010001000101101', '00000000010000001010001010000100', '10000010010001111001001000010000', '00000110100100101001000001010001', '00001000000000000000000010010000', '00000010010000000010001000000000', '00000000000000000100000000000000', '00000000100000000000000000000000', '00000000000000010000001000000000', '00100000010000000000000000000000', '00001000000000001000000100000100', '00000000000000000000000000000000', '00100000010100000000000000000000', '10100000000000000000000001000000', '00000000000010000001100000001010', '01100001001010101000000000100000']


Teraz jeszcze potrzeba nam tylko kilku pomocniczych funkcji:
1. into_int_array() - zamienia słowo na listę kodów ASCII
1. get_bin() - zamienia liczbę całkowitą na jej reprezentacje binarną odpowiedniej długości
1. little_endian_bstr() - zamienia listę liczb całkowitych na binarne liczby w modelu little endian
1. le_to_be() - zamienia liczby binarne zapisane w little endian na big endian

In [32]:
from textwrap import wrap

def split_to_16_32bit_words(message):
    return wrap(f'{message:x<64}', 4)[:16]

split = split_to_16_32bit_words("ToJestBardzoDlugiTekstMajacyMiec64ZnakiAleToDoscTrudnoOsiagalne")
print(split)
# oczekiwany wynik:
# [ 'ToJe', 'stBa', 'rdzo', 'Dlug',
#   'iTek', 'stMa', 'jacy', 'Miec',
#   '64Zn', 'akiA', 'leTo', 'Dosc',
#   'Trud', 'noOs', 'iaga', 'lnex']

def into_int_array(message):
    return list(map(ord, message))

print(into_int_array(split[0]))
# oczekiwany wynik:
# [84, 111, 74, 101]

def get_bin(num, blen):
    return format(num, 'b').zfill(blen)

print(get_bin(7, 4))
# oczekiwany wynik:
# "0111"

def little_endian_bstr(int_list):
    bin_str = ""
    for x in int_list: 
        bin_str = get_bin(x, 8) + bin_str
    return bin_str

print(little_endian_bstr([84, 111, 74, 101]))
# oczekiwany wynik:
# "01100101010010100110111101010100"

def le_to_be(bstr):
    wrapped = wrap(bstr, 8)
    wrapped.reverse()
    return "".join(wrapped)

print(le_to_be("01100101010010100110111101010100"))
# oczekiwany wynik:
# "01010100011011110100101001100101"

['ToJe', 'stBa', 'rdzo', 'Dlug', 'iTek', 'stMa', 'jacy', 'Miec', '64Zn', 'akiA', 'leTo', 'Dosc', 'Trud', 'noOs', 'iaga', 'lnex']
[84, 111, 74, 101]
0111
01100101010010100110111101010100
01010100011011110100101001100101


Pełna implementacja funkcji hashującej Salsa20/8 (funkcja hashująca Salsa20 o zredukowanej do 8 liczbie rund):

In [39]:
# Funkcja Haszująca Salsa20
# x to sekwencja 64 bajtów (np. 64 litery)
def salsa20_hash(x):
    # zamieniamy nasze dane wejściowe na 16 4 bajtowych słów
    x = split_to_16_32bit_words(x)
    x = list(map(into_int_array, x))
    x = list(map(little_endian_bstr, x))

    # Pierwsza runda
    z = list(map(double_round, x))
    # Pozostałe 7 rund naszej skróconej funkcji Salsa20/8
    for i in range(7):
        z = list(map(double_round, z))

    xhash = []
    # na koniec, do oryginalnych danych dodajemy hash,
    # a następnie zamieniamy wynik na big endian
    for elem1, elem2 in zip(z, x):
        xhash.append(le_to_be(bin_add(elem1, elem2)))
    
    # x to również sekwencja 64 bajtów, na nasze potrzeby 16 4 bajtowych grup
    return xhash


test_str = "ToJestBardzoDlugiTekstMajacyMiec64ZnakiAleToDoscTrudnoOsiagalne"
s20h = salsa20_hash(test_str)
print(s20h)
# oczekiwany wynik:
# [ '10101000110111101001010011001010', '11100110111010001000010011000010',
#   '11100100110010001111010011011110', '10001000110110001110101011001110',
#   '11010010101010001100101011010110', '11100110111010001001101011000010',
#   '11010100110000101100011011110010', '10011010110100101100101011000110',
#   '01101100011010001011010011011100', '11000010110101101101001010000010',
#   '11011000110010101010100011011110', '10001000110111101110011011000110',
#   '10101000111001001110101011001000', '11011100110111101001111011100110',
#   '11010010110000101100111011000010', '11011000110111001100101011110000']

['10101000110111101001010011001010', '11100110111010001000010011000010', '11100100110010001111010011011110', '10001000110110001110101011001110', '11010010101010001100101011010110', '11100110111010001001101011000010', '11010100110000101100011011110010', '10011010110100101100101011000110', '01101100011010001011010011011100', '11000010110101101101001010000010', '11011000110010101010100011011110', '10001000110111101110011011000110', '10101000111001001110101011001000', '11011100110111101001111011100110', '11010010110000101100111011000010', '11011000110111001100101011110000']


### Algorytm scryptBlockMix
Scrypt wykorzystuje `scryptBlockMix` jako funkcję haszującą.
Reprezentacja algorytmu:
<!-- Tłumaczenie? -->
```bash
Parameters:
  r       Block size parameter.

  Input:
      B[0] || B[1] || ... || B[2 * r - 1]
          Input octet string (of size 128 * r octets),
          treated as 2 * r 64-octet blocks,
          where each element in B is a 64-octet block.

  Output:
      B\'[0] || B\'[1] || ... || B\'[2 * r - 1]
        Output octet string.

  Steps:

    1. X = B[2 * r - 1]

    2. for i = 0 to 2 * r - 1 do
        T = X xor B[i]
        X = Salsa (T)
        Y[i] = X
      end for

    3. B\' = (Y[0], Y[2], ..., Y[2 * r - 2], Y[1],
              Y[3], ..., Y[2 * r - 1])
```

### Algorytm scryptROMix

Algorytm sROMix wykorzystuje scryptBlockMix jako funkcję haszującą `H` oraz funkcję `Integerify` (dosłowne tłumaczenie - ???)
<!-- Again tłumaczenie ? -->
Reprezentacja algorytmu:  
```bash
Input:
        r       Block size parameter.
        B       Input octet vector of length 128 * r octets.
        N       CPU/Memory cost parameter, must be larger than 1,
                a power of 2, and less than 2^(128 * r / 8).

Output:
        B\'      Output octet vector of length 128 * r octets.

Steps:

    1. X = B

    2. for i = 0 to N - 1 do
        V[i] = X
        X = scryptBlockMix (X)
    end for

    3. for i = 0 to N - 1 do
        j = Integerify (X) mod N
                where Integerify (B[0] ... B[2 * r - 1]) is defined
                as the result of interpreting B[2 * r - 1] as a
                little-endian integer.
        T = X xor V[j]
        X = scryptBlockMix (T)
    end for

    4. B\' = X
```



### Scrypt
Wracając do początku sekcji - algorytm można przedstawić w 3 krokach.
Zapis `PBKDF2-HMAC-SHA-256` oznacza, że używamy algorytmu `PBKDF2` (Password-Based Key Derivation Function - Opartej na haśle Funkcji Wyprowadzania Klucza) z `HMAC-SHA-256` jako `PRF` (Pseudorandom Function - Funkcja Pseudolosowa).

**Algorytm `scrypt`:**  

Dane wejściowe:
```
        P       Passphrase, an octet string.
        S       Salt, an octet string.
        N       CPU/Memory cost parameter, must be larger than 1,
                a power of 2, and less than 2^(128 * r / 8).
        r       Block size parameter.
        p       Parallelization parameter, a positive integer
                less than or equal to ((2^32-1) * hLen) / MFLen
                where hLen is 32 and MFlen is 128 * r.
        dkLen   Intended output length in octets of the derived
                key; a positive integer less than or equal to
                (2^32 - 1) * hLen where hLen is 32.
```

Wyjście:
```
        DK      Derived key, of length dkLen octets.
```

Kroki:
```
1.  Initialize an array B consisting of p blocks of 128 * r octets
        each:
            B[0] || B[1] || ... || B[p - 1] = PBKDF2-HMAC-SHA256 (P, S, 1, p * 128 * r)

2.  for i = 0 to p - 1 do
        B[i] = scryptROMix (r, B[i], N)
    end for

3.  DK = PBKDF2-HMAC-SHA256 (P, B[0] || B[1] || ... || B[p - 1], 1, dkLen)
```

#TODO Omówienie algorytmu? W którym miejscu wykazuje on własność Memmory Hardness.

### Atak kanałem bocznym na `scrypt`

`scrypt` jest wrażliwy na `Atak Czasowy na Cache`.

`Atak czasowy na Cache` - ten typ ataku polega na pozyskiwaniu informacji przez badanie dostępności lub niedostępności danych w cache procesora.
Jedną z metod przeprowadzenia tego ataku jest metoda Yuval'a Yarom'a `PRIME+PROBE` i jest najlepszą metodą, której można użyć przeciwko scryptowi.

`PRIME+PROBE` (dosł. Zastaw i Sonduj) - Metoda Ataku polegająca na celowym wypchnięciu danych ofiary z cache procesora (Zastaw) przez uzyskanie dostępu do danych, które atakujący wie, że spowodują wypchnięcie z cache danych ofiary. Następnie, atakujący czeka pewien czas dając ofierze uzyskać dostęp do swoich danych, po czym próbuje uzyskać dostęp do swoich danych (Sonduj). Jeżeli czas dostępu jest relatywnie krótki, to oznacza, że ofiara nie uzyskiwała dostępu do swoich danych, bo dane atakującego wciaż są dostępne w cache. Jeżeli czas dostępu jest dłuższy, oznacza to, że ofiara użyskała dostęp do badanych danych, ponieważ dane atakującego zostały zepchnięte do pamięci niższego poziomu.
Atak ten może zostać zastosowany w dowolnym momencie kiedy atakujący współdzieli z ofiarą jakiś poziom cache procesora. To może się zdarzyć kiedy atakujący i ofiara są procesami na tej samej maszynie, ale również kiedy atakujący i ofiara pracują na różnych maszynach wirtualnych, hostowanych na jednej maszynie fizycznej. Wystarczy współdzielenie przez nich cache procesora.

---

**Procedura ataku na `scrypt`:**  
To co sprawia, że `Atak Czasowy na Cache` jest możliwy to podany poniżej kod funkcji `scryptROMix`:
```
    j = Integerify (X) mod N
    T = X xor V[j]
    X = scryptBlockMix (T)
```

\# Najpierw ogarnę memory hardness, bo coś mi się tu nie kalkuluje XDDD

# Czym jest `yescrypt`?

`Yescrypt` jest funkcją wyprowadzania klucza opartą na haśle. Został stworzony na bazie `scrypt`.  
Yescrypt jest domyślną funkcją haszującą dla systemów Debian 11, Fedora 35+, Kali Linux 2021.1+ oraz Ubuntu 20.04+.

Zdaje się on być najlepiej skalującym algorytmem do haszowania. Oferuje prawie optymalne bezpieczeństwo ataków bruteforce kosztem złożoności.  
Warto mieć na uwadze, że w przypadku dużych implementacji będzie to stosunkowo mała część całkowitej autoryzacji. 

| Zalety | Wady |
| --- | --- |
| Lepsza ochrona przed atakami | Złożoność |
| Oparte na zaakceptowanych przez NIST funkcjach haszujących (HMAC, PBKDF2, SHA-256)| Podatne na atak kanałem bocznym (Cache-timing)|
| Czas działania można regulować w zależności od wykorzystania pamięci | Wspierany w mniejszej ilości technologi |

Yescrypt nie wygrał w [PHC](https://en.wikipedia.org/wiki/Password_Hashing_Competition). Zwycięzcą w 2015 roku został `Argon2`. 
Mimo to yescrypt dostał specjalne wyróżnienie (razem z `Catena`, `Lyra2`, `Makwa`).

Istnieje też `yespower`, który jest PoW (proof of work) zbudowanym na scrypcie. Zamiast wyprowadzać klucze i haszować hasła yespower jest przeznaczony do przetwarzania nagłówków bloków w `blockchain` (włączając wartość nounce).


# `Balloon Hashing`



Kolejna odnoga funkcji KDF. Autorzy zapewniają, że:
1. Algorytm wykazuje właściwości `memory-hardness`. 
1. Jest bazowany na podstawowych funkcjach (SHA-512 ...). Zatem może być używany w dowolnym standardzie.
1. Odporny na ataki kanłem bocznym (side-channel attacks). Dostęp do pamięci jest niezależny od danych, które są haszowane.
1. Łatwy w implementacji a złożoność obliczeniowa jest podobna do innych algorytmów tego sortu.


# Podsumowanie

|Własność|scrypt|yescrypt|baloon hashing|
|---|---|---|---|
|Memory-Hardness|Tak|Tak|Tak|
|Bazowana na 'Standard Primitives' (np. SHA-512 itd.)|Tak|Tak|Tak|
|Odporność na ataki Cache (kanałem bocznym)|Nie<sup>1</sup>|Nie|Tak|
|Złożoność|Ω(n<sup>2</sup>ω)||Ω(n<sup>2</sup>/log n)<sup>2</sup>|


<sup>1</sup>Zostało udowodnione, że funkcja ROMix jest sekwencyjnie odporna dla wyroczni losowej. Bezpieczeństwo scryptu opiera się na założeniu, że BlockMix() wykorzystujący Salsa20/8 nie wykazuje, żadnych "skrótów" które pozwoliłyby iterować łatwiej niż w wyroczni losowej.  
<sup>2</sup>W ustawieniu równoległym Ω(n<sup>2</sup>). Dla rundy (odpowiednie warunki)  Ω(n<sup>5/3</sup>)<sup>2</sup>. Funkcja wykazująca własność memory-hard Ω(n<sup>2</sup>/log n).  


### Wnioski
1. Warto używać yescrypta gdy chcemy bronić się przed atakami równoległymi ale dostęp do pamięci będzie zależny od hasła. Jest to słabe podejście jeśli adwersach może mieć dostęp do takiej informacji.  
1. Baloon Hashin jest przydatny w obronie przed atakami sekwencyjnymi. Jest dostęp do pamięci jest niezależny od hasła. Jest asymptotycznie słabszy przy dużych atakach równoległych.
[Źródło](https://eprint.iacr.org/2016/027.pdf)